<a href="https://colab.research.google.com/github/VivekanandaMudelli/hand_drawn_sketch_recognition/blob/main/KNN(Pca).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gdown
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
file_id = "1EFQlS9C4YGT5CNjD_NX_Q-HaiyLH4sjE"

# Construct the direct download URL
download_url = f"https://drive.google.com/uc?id={file_id}"

# Download the file
output = "data.csv"  # Change filename as needed
gdown.download(download_url, output, quiet=False)

# Read the CSV file
data = pd.read_csv(output)
data = data.dropna()

X = data.drop(columns = ['label'])
y = data['label']

print(X.shape)
print(y.shape)

Downloading...
From (original): https://drive.google.com/uc?id=1EFQlS9C4YGT5CNjD_NX_Q-HaiyLH4sjE
From (redirected): https://drive.google.com/uc?id=1EFQlS9C4YGT5CNjD_NX_Q-HaiyLH4sjE&confirm=t&uuid=63411329-fcfc-4a35-ade3-e8db8326e98b
To: /content/data.csv
100%|██████████| 943M/943M [00:14<00:00, 63.3MB/s]


(20180, 8100)
(20180,)


In [3]:
# Mean
X_mean = X.mean()

# Standard deviation
X_std = X.std()

# Standardization
Z = (X - X_mean) / X_std.replace(0,1e-10)

In [4]:
# Importing PCA
from sklearn.decomposition import PCA

# Taking top 300 dimensions
pca = PCA(n_components=300)
pca.fit(Z)
x_pca = pca.transform(Z)

# Create the dataframe
df = pd.DataFrame(x_pca,
                       columns=['PC{}'.
                       format(i+1)
                        for i in range(300)])
print(df)

             PC1        PC2        PC3        PC4        PC5        PC6  \
0     -13.473498  14.170485  -8.592970  -0.276517  -4.310716 -12.837159   
1      -7.392499  21.470417 -16.521424  -9.054980  10.826061   2.505688   
2     -11.364221   4.819343  -5.701316  -9.767068  -3.730639 -13.074415   
3      -9.464440   4.396213 -14.905356  -3.827590  -1.930120  -7.333155   
4      -3.514694   4.805839 -10.267748   6.538963  -6.552484   2.683344   
...          ...        ...        ...        ...        ...        ...   
20175  -3.464935   7.745460 -29.708590 -10.414634  14.356322   2.594481   
20176  -6.965887   8.512735  -5.489306   1.021765   0.134600  -6.489955   
20177 -10.654562  -4.086706  -9.890633  -5.559354   9.865369   8.734751   
20178   6.125788  -1.236722 -12.704955 -12.068881   0.344318  -0.449119   
20179  -3.484752  13.398968 -10.478203  -7.359177   4.313188  19.720090   

             PC7       PC8        PC9      PC10  ...     PC291     PC292  \
0       0.117438 -2.761

In [10]:
df['label'] = y

In [11]:
def distance(x,y):
  return np.linalg.norm(x-y)

In [12]:
def k_nearest_neighbors(p, points, k):
    dist = [distance(p,p1) for p1 in points]
    neighborId = np.argsort(dist)
    neighbor = [points[i] for i in neighborId[:k]]

    Knn = [neighbor,neighborId[:k]]
    return Knn

In [13]:
import statistics as st

def knn_predict(X_train, y_train, testPoint, k):
  a,b = k_nearest_neighbors(testPoint,X_train,k)# a maps to the list of points and b maps to the list of indices
  k_labels = [y_train[i] for i in b] #list of labels for the k nearest neighbors
  return st.mode(k_labels)

In [14]:
# import pandas as pd
# df = pd.read_csv('/content/pca_data.csv')
# df.head()

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm

X_new = df.drop('label', axis=1)
y_new = df['label']

In [16]:
X_train, X_val, y_train, y_val = train_test_split(X_new, y_new, train_size=0.7, random_state=42)
X_train=X_train.values
y_train=y_train.values
X_val=X_val.values
y_val=y_val.values
k=5
predictions=[]

In [17]:
for testPoint in tqdm(X_val, desc="Predicting", unit="sample"):
    predictions.append(knn_predict(X_train, y_train, testPoint, k))

Predicting: 100%|██████████| 6054/6054 [07:47<00:00, 12.94sample/s]


In [18]:
accuracy = accuracy_score(y_val, predictions)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

Validation Accuracy: 22.53%
